In [9]:
import geemap
geemap.update_package()
import geemap.foliumap as geemap
import ee
import geemap
import geemap.colormaps as cm
Map = geemap.Map(center=[40,-100], zoom=4)
Map
import pandas as pd

Unzipping geemap-master.zip ...
Data downloaded to: C:\Users\yoah2447\Downloads\geemap-master

Please comment out 'geemap.update_package()' and restart the kernel to take effect:
Jupyter menu -> Kernel -> Restart & Clear Output


# mapping landcover 2016 and state boundary

*note : 2010 boundary doesn't have all the states

In [47]:
Map = geemap.Map(center=[40, -100], zoom=4)
Map
#dem = ee.Image('USGS/SRTMGL1_003')
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2016_REL')
nlcd2016 = dataset.filter(ee.Filter.eq('system:index', '2016')).first();
landcover = nlcd2016.select('landcover');
Map.addLayer(landcover, {}, 'landcover')

# Map.addLayer(
#     landcover,
#     {'bands': ['landcover'], 'min': 0, 'max': 95},
#     'landcover',
# )
Map
i = 1
#county = ee.Image("TIGER/2016/Counties") #.filter(ee.Filter.eq('STATEFP', '23'));
#county = ee.FeatureCollection("TIGER/2020/TRACT").filter(ee.Filter.eq('STATEFP', '23'));
#county =ee.FeatureCollection("TIGER/2020/TABBLOCK20").filter(ee.Filter.eq('COUNTYFP20', '103')).filter(ee.Filter.eq('STATEFP20', '13'));
#county =ee.FeatureCollection("TIGER/2010/BG").filter(ee.Filter.eq('COUNTYFP10', DF['COUNTYFP'][i])).filter(ee.Filter.eq('STATEFP10',DF['STATEFP'][i]));
county =ee.FeatureCollection("TIGER/2010/BG")

Map.addLayer(county, {}, 'US States')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## mapping landcover 2019 and state boundary

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)

dataset = ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2019')
landcover = dataset.select('landcover')
Map.addLayer(landcover, {}, 'NLCD 2019')

states = ee.FeatureCollection("TIGER/2018/States")
Map.addLayer(states, {}, 'US States')
Map.add_legend(builtin_legend='NLCD')
Map

# zonal stat 

# census tract level (2010 census boundary)

In [4]:
DF = pd.read_csv('D:/DATA/NHGIS/nhgis0026_shape (1)/nhgis0026_shape/nhgis0026_shapefile_tl2010_us_tract_2010/US_tract_2010.csv')

DF['GEOID'] =DF['GEOID'].apply(lambda x: '{0:0>11}'.format(x))
DF

DF = pd.read_csv('./countyfips.csv')
DF['STATEFP'] =DF['STATEFP'].apply(lambda x: '{0:0>2}'.format(x))
DF['COUNTYFP'] =DF['COUNTYFP'].apply(lambda x: '{0:0>3}'.format(x))
DF['GEOID'] =DF['GEOID'].apply(lambda x: '{0:0>5}'.format(x))
DF

,GEOID,STATEFP,COUNTYFP
0,01107,01,107
1,01065,01,065
2,01125,01,125
3,01055,01,055
4,01015,01,015
...,...,...,...
3228,72061,72,061
3229,72127,72,127
3230,78030,78,030
3231,78020,78,020


In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)

dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2016_REL')
nlcd2016 = dataset.filter(ee.Filter.eq('system:index', '2016')).first();
landcover = nlcd2016.select('landcover');
# Map.addLayer(landcover, {}, 'landcover')
# i=1
# states = ee.FeatureCollection("TIGER/2010/BG").filter(ee.Filter.eq('COUNTYFP10', DF['COUNTYFP'][i])).filter(ee.Filter.eq('STATEFP10',DF['STATEFP'][i]));
# Map.addLayer(states, {}, 'US States')
# Map.add_legend(builtin_legend='NLCD')
# Map


for i in range(0,len(DF)):
    county = ee.FeatureCollection("TIGER/2010/Tracts_DP1").filter(ee.Filter.eq('geoid10', DF['GEOID'][i]));
    #county =ee.FeatureCollection("TIGER/2020/TABBLOCK20").filter(ee.Filter.eq('COUNTYFP20', DF['COUNTYFP'][i])).filter(ee.Filter.eq('STATEFP20',DF['STATEFP'][i]));
    #county =ee.FeatureCollection("TIGER/2010/BG").filter(ee.Filter.eq('COUNTYFP10', DF['COUNTYFP'][i])).filter(ee.Filter.eq('STATEFP10',DF['STATEFP'][i]));

    nlcd_stats = './censustract2010boundary/lu_stats'+str(DF['GEOID'][i])+'.csv'
    try:
        geemap.zonal_statistics_by_group(
            landcover,
            county,
            nlcd_stats,
            statistics_type='SUM',
            denominator=1000000,
            decimal_places=2,
        )
    except Exception:
        pass

# census tract level (2020 census boundary)

In [7]:
DF = pd.read_csv('./countyfips.csv')
DF['STATEFP'] =DF['STATEFP'].apply(lambda x: '{0:0>2}'.format(x))
DF['COUNTYFP'] =DF['COUNTYFP'].apply(lambda x: '{0:0>3}'.format(x))
DF.loc[932]
DF.tail


<bound method NDFrame.tail of       GEOID STATEFP COUNTYFP
0      1107      01      107
1      1065      01      065
2      1125      01      125
3      1055      01      055
4      1015      01      015
...     ...     ...      ...
3228  72061      72      061
3229  72127      72      127
3230  78030      78      030
3231  78020      78      020
3232  78010      78      010

[3233 rows x 3 columns]>

In [ ]:
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2016_REL')
nlcd2016 = dataset.filter(ee.Filter.eq('system:index', '2016')).first();
landcover = nlcd2016.select('landcover');


for i in range(0,len(DF)):
    county = ee.FeatureCollection("TIGER/2020/TRACT").filter(ee.Filter.eq('STATEFP', fips));
    #county =ee.FeatureCollection("TIGER/2020/TABBLOCK20").filter(ee.Filter.eq('COUNTYFP20', DF['COUNTYFP'][i])).filter(ee.Filter.eq('STATEFP20',DF['STATEFP'][i]));
    #county =ee.FeatureCollection("TIGER/2010/BG").filter(ee.Filter.eq('COUNTYFP10', DF['COUNTYFP'][i])).filter(ee.Filter.eq('STATEFP10',DF['STATEFP'][i]));

    nlcd_stats = './censustract2020boundary/lu_stats'+DF['STATEFP'][i]+ DF['COUNTYFP'][i]+'.csv'
    try:
        geemap.zonal_statistics_by_group(
            landcover,
            county,
            nlcd_stats,
            statistics_type='SUM',
            denominator=1000000,
            decimal_places=2,
        )
    except Exception:
        pass

# Census block level (2010 census boundary) 

In [8]:
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2016_REL')
nlcd2016 = dataset.filter(ee.Filter.eq('system:index', '2016')).first();
landcover = nlcd2016.select('landcover');

for i in range(3000,len(DF)):
    #county = ee.FeatureCollection("TIGER/2020/TRACT").filter(ee.Filter.eq('STATEFP', fips));
    #county =ee.FeatureCollection("TIGER/2020/TABBLOCK20").filter(ee.Filter.eq('COUNTYFP20', DF['COUNTYFP'][i])).filter(ee.Filter.eq('STATEFP20',DF['STATEFP'][i]));
    county =ee.FeatureCollection("TIGER/2010/BG").filter(ee.Filter.eq('COUNTYFP10', DF['COUNTYFP'][i])).filter(ee.Filter.eq('STATEFP10',DF['STATEFP'][i]));

    nlcd_stats = './Blocklevel_2010boundary/lu_stats'+DF['STATEFP'][i]+ DF['COUNTYFP'][i]+'.csv'
    try:
        geemap.zonal_statistics_by_group(
            landcover,
            county,
            nlcd_stats,
            statistics_type='SUM',
            denominator=1000000,
            decimal_places=2,
        )
    except Exception:
        pass

Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing 

KeyboardInterrupt: 

# census block level (2020 census boundary)

In [48]:
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2016_REL')
nlcd2016 = dataset.filter(ee.Filter.eq('system:index', '2016')).first();
landcover = nlcd2016.select('landcover');


for i in range(0,len(DF)):
    #county = ee.FeatureCollection("TIGER/2020/TRACT").filter(ee.Filter.eq('STATEFP', fips));
    county =ee.FeatureCollection("TIGER/2020/TABBLOCK20").filter(ee.Filter.eq('COUNTYFP20', DF['COUNTYFP'][i])).filter(ee.Filter.eq('STATEFP20',DF['STATEFP'][i]));
    #county =ee.FeatureCollection("TIGER/2010/BG").filter(ee.Filter.eq('COUNTYFP10', DF['COUNTYFP'][i])).filter(ee.Filter.eq('STATEFP10',DF['STATEFP'][i]));

    nlcd_stats = './Blocklevel_2020boundary/lu_stats'+DF['STATEFP'][i]+ DF['COUNTYFP'][i]+'.csv'
    try:
        geemap.zonal_statistics_by_group(
            landcover,
            county,
            nlcd_stats,
            statistics_type='SUM',
            denominator=1000000,
            decimal_places=2,
        )
    except Exception:
        pass

Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats01107.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats01065.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats01125.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats01055.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats01015.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats01051.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\D

Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats01071.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats01017.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats01081.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats01113.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats01067.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats01045.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\D

Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats04009.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats04007.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats04017.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats04005.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats04025.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats04027.csv
Computing ... 
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\U

Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats05093.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats05143.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats05007.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats05087.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats05033.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats05131.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\D

Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats06023.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats06045.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats06033.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats06007.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats06053.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats06109.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Ele

...

Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats51550.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats51700.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats51830.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats51047.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats51157.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats51061.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\D

Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats54027.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats54003.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats54037.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats72147.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats72049.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats72071.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\D

Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats72059.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats72001.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats72111.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats72113.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats72075.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats72149.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Ele

Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats72033.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats72053.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats72089.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats72037.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats72103.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats72115.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Ele

Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats72105.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats72047.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats72021.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats72007.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats72061.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\Documents\02_Elevation\Blocklevel_2020boundary\lu_stats72127.csv
Computing ... 
Generating URL ...
Please wait ...
Data downloaded to C:\Users\yoah2447\D

## combine all files

In [ ]:
import glob 
import pandas as pd
files = [i for i in glob.glob('C:/Users/yoah2447/Documents/02_Elevation/Blocklevel_2020boundary/lu_'+'*.csv') ]
df = pd.concat([pd.read_csv(f) for f in files], axis = 0)
df.to_csv( 'C:/Users/yoah2447/Documents/02_Elevation/lu_stats_censustractlevel2020'+'.csv', index=False)